# AoF Replication 1 Solutions

### Task 1: Import and prepare data

Task 1.1: The file webstar_ready.csv is part of the AoE GitHub repository. You should see it in the AoE/kruger folder. In the cell below issue the required command to read in the data into Stata. You can use the help _commandname_ command to see the Stata documentation if you need a reminder of the correct syntax in the extra command cell provided.

In [ ]:
*** BEGIN SOLUTION
import delimited webstar_ready.csv, varn(1) clear
*** END SOLUTION

Task 1.2: Assign labels to the variable cltype. Name the label cltype_lab.

In [ ]:
*** BEGIN SOLUTION
label define cltype_lab 1 "Small" 2 "Regular" 3 "Regular/Aide", modify
label values cltype cltype_lab
*** END SOLUTION

### Task 2: Table I

Task 2.1: Run the test that compares the means of the Small, Regular and Regular/Aide class types for the variable pscore

In [ ]:
*** BEGIN SOLUTION
oneway pscore cltype
*** END SOLUTION

Task 2.2 Save the p-value for the F test with the respective degrees of freedom and F statistic produced by the previous command in the global macro task22

In [ ]:
*** BEGIN SOLUTION
global task22 = Ftail(`r(df_m)', `r(df_r)', `r(F)')
*** END SOLUTION

Task 2.3: Using a loop declare 4 new empty matrices named p_val_0 - p_val_3 to store the p-values of the F tests for each cohort for the variables free_lunch white sage cs pscore along the rows.

In [ ]:
*** BEGIN SOLUTION
forv i = 0/3 {
matrix p_val_`i' = J(5,1,.)
}
*** END SOLUTION

Task 2.4: Declare two local macros: rows containing the five variables free_lunch white sage cs pscore and cohs containing the values the variable cohort takes. Using two nested loops over cohorts and over variables fill the matrices created in task 2.3 with the respective p-values.

In [ ]:
*** BEGIN SOLUTION
loc rows free_lunch white sage cs pscore
loc cohs k 1 2 3
forvalues j = 1/4 {
local coh: word `j' of `cohs'
forvalues i = 1/5 {
local var: word `i' of `rows'
qui oneway `var' cltype if cohort == "`coh'"
* we need the right tail p-value
loc c = `j' - 1
matrix p_val_`c'[`i',1] = Ftail(`r(df_m)', `r(df_r)', `r(F)')
}
* matlist p_val_`c'
}
*** END SOLUTION

Task 2.5: Using a loop over cohorts, save the means of the same 5 variables (free_lunch white sage cs pscore along the rows) in 4 matrices named ave_0 - ave_3 for each cohort. Note that we need to declare any local macros every time we run a chunk of code, since they do not stay in Stata's memory. You may need to first install the user-written command tabstatmat.

In [ ]:
*** BEGIN SOLUTION
ssc install tabstatmat
loc cohs k 1 2 3
forvalues j = 1/4 {
loc c = `j' - 1
local coh: word `j' of `cohs'
qui tabstat free_lunch white sage cs pscore if cohort == "`coh'", by(cltype) save
tabstatmat res, nototal
matrix ave_`c' = res'
*matlist ave_`c'
}
*** END SOLUTION

### Task 3: Table V

Task 3.1: Run the appropriate regression to produce column (1) of Table V for the kindergarden cohort.

In [ ]:
*** BEGIN SOLUTION
reg pscore ib(2).cltype if cohort == "k", cluster(classid)
*** END SOLUTION

Task 3.2: Using the user-written suite of commands estout, run, store and report in the Stata resutls window, for the kindergarden cohort, columns (1) through (3) of Table V.

In [ ]:
ssc install estout
eststo clear
eststo: qui reg pscore ib(2).cltype if cohort == "k", cluster(classid)
estadd local fixed "No" , replace
eststo: qui reg pscore ib(2).cltype i.schidkn  if cohort == "k", cluster(classid)
estadd local fixed "Yes" , replace
eststo: qui reg pscore ib(2).cltype i.white i.female i.free_lunch i.schidkn if cohort == "k", cluster(classid)
estadd local fixed "Yes" , replace
esttab , replace nonumber nodepvar nonotes eqlabels(none) title("A. Kindergarten") se s(fixed N r2_a, label("School fixed effects" "Obs" "R-sq") fmt(0 %9.0fc 3 )) drop(_cons *0.* 2.cltype *schidkn*) mtitles ("(1)" "(2)" "(3)") star(* 0.1 ** 0.05 *** 0.01) varlabels(1.cltype "Small class" 3.cltype "Regular/aide class" 1.white "White/Asian" 1.female "Girl" 1.free_lunch "Free lunch") note("Standard errors (clustered by class) below coefficients." "Significance stars denote: * 0.1 ** 0.05 *** 0.01")
